12-2. hi hello training

In [2]:
# Lab 12 RNN
import numpy as np
import tensorflow as tf

# text: hihello
idx2char = ['h', 'i', 'e', 'l', 'o'] # voc index: h:0, i:1, e:2, l: 3, o:4
# Teach hello: hihell -> ihello
# x_data = [[0, 1, 0, 2, 3, 3]]  # hihell
y_data = [[1, 0, 2, 3, 3, 4]]  # ihello

num_classes = 5 # 입력개수(문자개수)
input_dim = 5  # one-hot size, same as hidden_size to directly predict one-hot
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

# hihell 문자열을 원핫 인코딩한 형태
x_one_hot = np.array([[[1, 0, 0, 0, 0],    # h 0
                       [0, 1, 0, 0, 0],    # i 1
                       [1, 0, 0, 0, 0],    # h 0
                       [0, 0, 1, 0, 0],    # e 2
                       [0, 0, 0, 1, 0],    # l 3
                       [0, 0, 0, 1, 0]]],  # l 3
                     dtype=np.float32)

y_one_hot = tf.keras.utils.to_categorical(y_data, num_classes=num_classes)
print(x_one_hot.shape)
print(y_one_hot.shape)

tf.model = tf.keras.Sequential()

# RNN model
# make cell and add it to RNN layer
# input_shape = (1,6,5) => number of sequence (batch), length of sequence, size of input dim
cell = tf.keras.layers.LSTMCell(units=num_classes, input_shape=(sequence_length, input_dim)) # input size:5
tf.model.add(tf.keras.layers.RNN(cell=cell, return_sequences=True)) # RNN 레이어 생성

# single LSTM layer can be used as well instead of creating LSTMCell
# tf.model.add(tf.keras.layers.LSTM(units=num_classes, input_shape=(sequence_length, input_dim), return_sequences=True))

# fully connected layer
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])

# train
tf.model.fit(x_one_hot, y_one_hot, epochs=50)
tf.model.summary()

predictions = tf.model.predict(x_one_hot)
for i, prediction in enumerate(predictions):
    print(prediction)
    # argmax: 가장 확률이 높은 인덱스 선택, idx2char: 문자로 변환
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))

(1, 6, 5)
(1, 6, 5)
Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 1.6101 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 3ms/step - loss: 1.6088 - accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 3ms/step - loss: 1.6075 - accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 3ms/step - loss: 1.6063 - accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 3ms/step - loss: 1.6050 - accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 3ms/step - loss: 1.6038 - accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 1.6026 - accuracy: 0.5000
Epoch 8/50
1/1 [==============================] - 0s 3ms/step - loss: 1.6013 - accuracy: 0.5000
Epoch 9/50
1/1 [==============================] - 0s 4ms/step - loss: 1.6001 - accuracy: 0.5000
Epoch 10/50
1/1 [==============================] - 0s 3ms/step - loss: 1.5989 - accuracy: 0.5000
Epoch 11/50
1/1 [===

12-3. long sequence RNN

In [ ]:
# Lab 12 Character Sequence RNN
import tensorflow as tf
import numpy as np

sample = " if you want you"
idx2char = list(set(sample))  # unique 문자만 뽑음
char2idx = {c: i for i, c in enumerate(idx2char)}  # index 부여

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

x_one_hot_eager = tf.one_hot(x_data, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
x_one_hot_numpy = tf.keras.utils.to_categorical(x_data, num_classes)  # it'll generate numpy array, either way works
y_one_hot_eager = tf.one_hot(y_data, num_classes)

tf.model = tf.keras.Sequential();
tf.model.add(tf.keras.layers.
             LSTM(units=num_classes, input_shape=(sequence_length, x_one_hot_eager.shape[2]), return_sequences=True))
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))
tf.model.summary()
tf.model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                 metrics=['accuracy'])
tf.model.fit(x_one_hot_eager, y_one_hot_eager, epochs=50)


predictions = tf.model.predict(x_one_hot_eager)

for i, prediction in enumerate(predictions):
    # print char using argmax, dict
    result_str = [idx2char[c] for c in np.argmax(prediction, axis=1)]
    print("\tPrediction str: ", ''.join(result_str))